In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

In [2]:
df = pd.read_csv('562_Dataset.csv')


In [3]:
county_name = list(set(df['county_name']))
county_id = {}
for i in range(len(county_name)):
    county_id[county_name[i]] = i
state_name = list(set(df['state']))
state_id = {}
for i in range(len(state_name)):
    state_id[state_name[i]] = i
region_name = list(set(df['region']))
region_id = {}
for i in range(len(region_name)):
    region_id[region_name[i]] = i
#county_id
    

In [4]:
df["county_name"].replace(county_id, inplace=True)
df["state"].replace(state_id, inplace=True)  
df["region"].replace(region_id, inplace=True) 

In [5]:
df.dropna(axis=0,inplace = True)

In [6]:
df.isnull().any()

caseid           False
race             False
vote2016         False
faminc           False
fips             False
wait             False
land_area        False
county_name      False
state            False
county_pop       False
income_county    False
region           False
dtype: bool

In [7]:
df = df[df.vote2016 != 5] #delete I don't recall candidate
df = df[df.vote2016 != 4]  # delete i did not cast a vote, meaningless
df = df[df.vote2016 != 3] #delete someone else - increases accuracy
#df = df[df.vote2016 != 2] #delete someone else - increases accuracy
df = df[df.wait != 6]  #delete i don't remember wait time
df = df[df.race != 7]  #delete i don't remember wait time
df = df.drop(['caseid', 'fips','county_name','wait'], axis=1) 
#caseid is useless, fips is useless according to documentation, 
#region is redundent we have states, county_name is confusing -> different states could have same county name

In [34]:
df

,race,vote2016,faminc,land_area,state,county_pop,income_county,region
2,1,1.0,9.0,594.44373,47,55200,41.618,2
3,1,1.0,6.0,1589.79370,47,208107,45.596,2
4,1,1.0,6.0,1589.79370,47,208107,45.596,2
5,1,2.0,12.0,1589.79370,47,208107,45.596,2
6,2,2.0,2.0,1589.79370,47,208107,45.596,2
...,...,...,...,...,...,...,...,...
30754,1,1.0,8.0,2081.72080,24,20609,40.280,1
30755,1,2.0,5.0,2081.72080,24,20609,40.280,1
30757,1,1.0,10.0,2081.72080,24,20609,40.280,1
30758,1,1.0,3.0,2081.72080,24,20609,40.280,1


In [31]:
df_1 = df[df.vote2016 == 1] #delete someone else - increases accuracy
df_1



,race,vote2016,faminc,land_area,state,county_pop,income_county,region
2,1,1.0,9.0,594.44373,47,55200,41.618,2
3,1,1.0,6.0,1589.79370,47,208107,45.596,2
4,1,1.0,6.0,1589.79370,47,208107,45.596,2
7,1,1.0,6.0,1589.79370,47,208107,45.596,2
8,1,1.0,12.0,1589.79370,47,208107,45.596,2
...,...,...,...,...,...,...,...,...
30746,1,1.0,5.0,2523.36120,24,30012,58.223,1
30754,1,1.0,8.0,2081.72080,24,20609,40.280,1
30757,1,1.0,10.0,2081.72080,24,20609,40.280,1
30758,1,1.0,3.0,2081.72080,24,20609,40.280,1


In [33]:
df_2 = df[df.vote2016 == 2] #delete someone else - increases accuracy
df_2

,race,vote2016,faminc,land_area,state,county_pop,income_county,region
5,1,2.0,12.0,1589.7937,47,208107,45.596,2
6,2,2.0,2.0,1589.7937,47,208107,45.596,2
11,1,2.0,11.0,1589.7937,47,208107,45.596,2
22,1,2.0,6.0,1589.7937,47,208107,45.596,2
24,1,2.0,4.0,1589.7937,47,208107,45.596,2
...,...,...,...,...,...,...,...,...
30750,1,2.0,6.0,10426.9800,24,44117,53.145,1
30751,1,2.0,9.0,10426.9800,24,44117,53.145,1
30752,1,2.0,7.0,10426.9800,24,44117,53.145,1
30753,1,2.0,4.0,10426.9800,24,44117,53.145,1


In [8]:
X = df.loc[:, df.columns != 'vote2016']
y = df.loc[:,df.columns == 'vote2016']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.20, random_state = 1)

In [9]:
lm_2 = linear_model.LogisticRegression(multi_class='ovr', solver='newton-cg')
lm_2.fit(X_train, y_train)
#.         accuracy.     f1
#newton-cg: 0.61.       0.61
#sag:        0.53.      0.11
#liblinear:  0.59.      0.54
#saga:      0.52.       0.05
#default:   0.59.       0.55

/Users/katemao/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/katemao/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/katemao/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/katemao/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/katemao/opt/anaconda3/lib/python3.8/sit

LogisticRegression(multi_class='ovr', solver='newton-cg')

In [10]:
lm_2.score(X_test, y_test)

0.632688927943761

In [11]:
print(metrics.classification_report(y_test, lm_2.predict(X_test)))

              precision    recall  f1-score   support

         1.0       0.61      0.68      0.64      2496
         2.0       0.66      0.59      0.62      2625

    accuracy                           0.63      5121
   macro avg       0.63      0.63      0.63      5121
weighted avg       0.64      0.63      0.63      5121



In [13]:
from sklearn.metrics import confusion_matrix

In [14]:
tn, fp, fn, tp = confusion_matrix(y_test, lm_2.predict(X_test)).ravel()
print('multi class logistic regression tn is ', tn)
print('multi class logistic regression fp is ', fp)
print('multi class logistic regression fn is ', fn)
print('multi class logistic regression tp is ', tp)

multi class logistic regression tn is  1698
multi class logistic regression fp is  798
multi class logistic regression fn is  1083
multi class logistic regression tp is  1542


In [15]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

In [16]:
# Build a Gaussian Random Forest Classifier
randomFor = RandomForestClassifier(n_estimators=100)

# Train model on training data
randomFor.fit(X_train, y_train)

randomFor.score(X_test, y_test)

<ipython-input-16-f9218f8d3190>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  randomFor.fit(X_train, y_train)


0.6221441124780316

In [17]:
print(metrics.classification_report(y_test, randomFor.predict(X_test)))

              precision    recall  f1-score   support

         1.0       0.61      0.61      0.61      2496
         2.0       0.63      0.63      0.63      2625

    accuracy                           0.62      5121
   macro avg       0.62      0.62      0.62      5121
weighted avg       0.62      0.62      0.62      5121



In [18]:
tn, fp, fn, tp = confusion_matrix(y_test, randomFor.predict(X_test)).ravel()
print('random forest tn is ', tn)
print('random forest fp is ', fp)
print('random forest fn is ', fn)
print('random forest tp is ', tp)

random forest tn is  1530
random forest fp is  966
random forest fn is  969
random forest tp is  1656


In [19]:
from sklearn.neighbors import KNeighborsClassifier
# Build a K-neighbors classifier
k_neigh = KNeighborsClassifier()

# Train model on training data
k_neigh.fit(X_train, y_train)

k_neigh.score(X_test, y_test)

/Users/katemao/opt/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.597734817418473

In [20]:
print(metrics.classification_report(y_test, k_neigh.predict(X_test)))

              precision    recall  f1-score   support

         1.0       0.58      0.61      0.60      2496
         2.0       0.61      0.59      0.60      2625

    accuracy                           0.60      5121
   macro avg       0.60      0.60      0.60      5121
weighted avg       0.60      0.60      0.60      5121



In [21]:
tn, fp, fn, tp = confusion_matrix(y_test, k_neigh.predict(X_test)).ravel()
print('k neighbours tn is ', tn)
print('k neighbours fp is ', fp)
print('k neighbours fn is ', fn)
print('k neighbours tp is ', tp)

k neighbours tn is  1517
k neighbours fp is  979
k neighbours fn is  1081
k neighbours tp is  1544


In [22]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm

In [23]:

clf_3 = svm.SVC()
clf_3.fit(X_train, y_train)
clf_3.score(X_test, y_test)


/Users/katemao/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5953915250927553

In [24]:
print(metrics.classification_report(y_test, clf_3.predict(X_test)))

              precision    recall  f1-score   support

         1.0       0.59      0.57      0.58      2496
         2.0       0.60      0.62      0.61      2625

    accuracy                           0.60      5121
   macro avg       0.59      0.59      0.59      5121
weighted avg       0.60      0.60      0.60      5121



In [30]:
tn, fp, fn, tp = confusion_matrix(y_test, clf_3.predict(X_test)).ravel()
print('clf_3 tn is ', tn)
print('clf_3 fp is ', fp)
print('clf_3 fn is ', fn)
print('clf_3 tp is ', tp)

clf_3 tn is  1411
clf_3 fp is  1085
clf_3 fn is  987
clf_3 tp is  1638


In [26]:
from sklearn.ensemble import VotingRegressor

In [27]:
lm4 = linear_model.LogisticRegression(multi_class='ovr', solver='newton-cg')
randomFor2 = RandomForestClassifier(n_estimators=100)

In [28]:
er = VotingRegressor([('lr', lm4), ('rf', randomFor2)])

In [29]:
er.fit(X_train, y_train)

/Users/katemao/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_voting.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: The estimator LogisticRegression should be a regressor.

In [ ]:
abstract, introduction, related works, 